In [2]:
from flamebench import ConfigParser
from flamebench.data_sampler import ZeroDSampler, OneDSampler
from flamebench.dataset_tools import DatasetMerger, DataAugmenter
from flamebench.dataset_tools.container import Container
from flamebench.nn_framework.model import MLPModel
from flamebench.utils import visualiser

import torch
import torch.nn as nn
import numpy as np

data sample

In [3]:
sampler = OneDSampler()
sampler.sample()
data = sampler.get_data()
sampler.save()
sampler.clean()

Loaded config for fuel: H2

 Starting sampling pipeline...
Calculating steady flame properties with Cantera...
Converted to relative path for Cantera: mechanisms/Burke2012_s9r23.yaml
Laminar Flame Speed      :   2.6543095776 m/s
Laminar Flame Thickness  :   0.0029160460 m
Flame Thickness: 2.92e-03
Flame Speed: 2.65e+00
Domain Width: 1.46e-02
Domain Length: 1.46e-01
Half Domain Length: 7.29e-02
Target Time Step: 1.00e-06
Chemical Time Scale: 1.10e-03
Sample Time Steps: 1.00e+02
Estimated Time Step: 1.10e-04
Estimated Sim Time: 1.10e-02
Estimated Write Time Step: 1.10e-04
SUCCESS Updated 1D sample configuration (system/controlDict)
SUCCESS Created 0/ species files
SUCCESS Updated setFieldsDict
SUCCESS Updated CanteraMechanismFile to mechanisms/Burke2012_s9r23.yaml
⚙️ Running Allrun script...


FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [ ]:
config_0d = ConfigParser.load_config("config/0d_config.yaml")
sampler_0d = ZeroDSampler(config_0d)
sampler_0d.sample()
data_0d = sampler_0d.data()

config_1d = ConfigParser.load_config("config/1d_config.yaml")
sampler_1d = OneDSampler(config_1d)
sampler_1d.sample()
data_1d = sampler_1d.data

data preprocess

In [ ]:
merged_data = DatasetMerger.merge([data_0d, data_1d])
augmented_data = DataAugmenter.add_multi_scale_perturbation(merged_data)
# other data processing operations can be added here

data train

In [ ]:
featureSize = 10
labelSize = 1 # Constrained to one when using default Container
totalRecords = 1000

epochs: int = 10
batch_size: int = 64
lr: float = 1e-3

# Define data and feed into Container
data: np.ndarray = np.random.random((totalRecords, featureSize+labelSize))
dataContainer: Container = Container(data)

# Shuffle and split data
dataContainer.shuffle()
container_train, container_vali = dataContainer.train_test_split(0.8)

# Create and train model
model: MLPModel = MLPModel([featureSize, 64, 64, labelSize])
model.set_optim(torch.optim.Adam(model.parameters(), lr=lr))
train_loss, val_loss = model.fit(container_train, container_vali, epochs, batch_size, verbose=True)

# TODO: val_loss is not list[float] but list[list[float]] as there may be multiple metrics.
visualiser.plot_loss_curve(train_loss, val_loss)